In [ ]:
#import R programs
library("phyloseq")
library("vegan")
library("dplyr")
library("microbiome") 
library("pairwiseAdonis"); packageVersion("pairwiseAdonis")

In [ ]:
#setworking directory
setwd("")

In [ ]:
#import files
SV_T <- t(read.table("export_TL_T/table-BacArc_TL_TFill.txt", 
                     row.names = 1, header = TRUE,
                    stringsAsFactors =TRUE))
dim(SV_T)

SV_T_DLAB <- t(read.table("export_TL_T_DLAB/table-BacArc_TL_DLAB.txt", 
                     row.names = 1, header = TRUE,
                    stringsAsFactors =TRUE))

SV_T_DSTOK <- t(read.table("export_TL_DSTOK/table-BacArc_TL_DSTOK.txt", 
                     row.names = 1, header = TRUE,
                    stringsAsFactors =TRUE))

SV_T_SINT <- t(read.table("export_TL_SINT/table-BacArc_TL_SINT.txt", 
                     row.names = 1, header = TRUE,
                    stringsAsFactors =TRUE))
SV_T_MMEA <- t(read.table("export_TL_MMEA/table-BacArc_TL_MMEA.txt", 
                     row.names = 1, header = TRUE,
                    stringsAsFactors =TRUE))

SV_T_DLAB_AH <- t(read.table("export_TL_DLAB_AH/table-BacArc_TL_DLAB_AH.txt", 
                     row.names = 1, header = TRUE,
                    stringsAsFactors =TRUE))

SV_T_DSTOK_AH <- t(read.table("export_TL_DSTOK_AH/table-BacArc_TL_DSTOK_AH.txt", 
                     row.names = 1, header = TRUE,
                    stringsAsFactors =TRUE))

SV_T_SINT_AH <- t(read.table("export_TL_SINT_AH/table-BacArc_TL_SINT_AH.txt", 
                     row.names = 1, header = TRUE,
                    stringsAsFactors =TRUE))


SV_S <- t(read.table("export_TL_S/table-BacArc_TL_SFill.txt", 
                     row.names = 1, header = TRUE))
dim(SV_S)

SV_W <- t(read.table("export_TL_W/table-BacArc_TL_WFill.txt", 
                     row.names = 1, header = TRUE))
head(SV_W)





map<- read.table("metadata_TL16S.txt", 
                 row.names = 1, header = TRUE)
head(map)

## to import taxonomy I had to convert ; into tabs and remove last columns
tax <-as.matrix(read.table("exported_TL/taxonomy.tsv",  
                           row.names = 1, header = FALSE, sep = "\t"))
#head(tax)
tree_file <-read_tree("exported_TL/tree.nwk")
#head(tree_file)

colnames(tax) <- c ("Kingdom", "Phylum", "Class", "Order", "Family", "Genus", "Species", "asv", "label")
tax[tax==""] <- NA
head(tax)
dim(tax)


In [ ]:
#Convert files into phyloseq objects

#import tissue samples
ps_T = phyloseq(otu_table(SV_T, taxa_are_rows=FALSE), 
               sample_data(map), 
               tax_table(tax))
ps_T


ps_T_DLAB=phyloseq(otu_table(SV_T_DLAB, taxa_are_rows=FALSE), 
               sample_data(map), 
               tax_table(tax))
ps_T_DLAB

ps_T_DSTOK=phyloseq(otu_table(SV_T_DSTOK, taxa_are_rows=FALSE), 
               sample_data(map), 
               tax_table(tax))
ps_T_DSTOK

ps_T_SINT=phyloseq(otu_table(SV_T_SINT, taxa_are_rows=FALSE), 
               sample_data(map), 
               tax_table(tax))
ps_T_SINT


ps_T_OP_MMEA=phyloseq(otu_table(SV_T_MMEA, taxa_are_rows=FALSE), 
               sample_data(map), 
               tax_table(tax))
ps_T_OP_MMEA

#import AH tissue samples
ps_T_AH_DLAB=phyloseq(otu_table(SV_T_DLAB_AH, taxa_are_rows=FALSE), 
               sample_data(map), 
               tax_table(tax))
ps_T_AH_DLAB

ps_T_AH_DSTOK=phyloseq(otu_table(SV_T_DSTOK_AH, taxa_are_rows=FALSE), 
               sample_data(map), 
               tax_table(tax))
ps_T_AH_DSTOK

ps_T_AH_SINT=phyloseq(otu_table(SV_T_SINT_AH, taxa_are_rows=FALSE), 
               sample_data(map), 
               tax_table(tax))
ps_T_AH_SINT

#import sediment samples

ps_S = phyloseq(otu_table(SV_S, taxa_are_rows=FALSE), 
               sample_data(map), 
               tax_table(tax))
ps_S

#import water samples
ps_W = phyloseq(otu_table(SV_W, taxa_are_rows=FALSE), 
               sample_data(map), 
               tax_table(tax))
ps_W

In [ ]:
#subset water samples from LooeKey collections
ps_W_OR =subset_samples(ps_W, Experiment=="original")
ps_W_OR

ps_W_OP =subset_samples(ps_W, Experiment=="oppurtunistic")
ps_W_OP

In [ ]:
#subset water samples from LooeKey collections
ps_W_OR =subset_samples(ps_W, Experiment=="original")
ps_W_OR

ps_W_OP =subset_samples(ps_W, Experiment=="oppurtunistic")
ps_W_OP

#remove MMEA from other tissue samples
ps_T_OR = subset_samples(ps_T, SpeciesCode!="MMEA")
ps_T_OR
#combine sediment, water, and tissue from the original dataset
ps_merge_OR= merge_phyloseq(ps_W_OR, ps_S, ps_T_OR)
#combine all sediment, water, and tissue samples
ps_merge= merge_phyloseq(ps_W, ps_S, ps_T)
ps_merge

In [ ]:
#Filter ASV abundances
ps_fill_T_OR = filter_taxa(ps_T_OR, function(x) sum(x > 5 ) > (0.15*length(x)), TRUE)
ps_fill_T_OR

In [ ]:
#CLR transformation
ps_clr_T_OR <- microbiome::transform(ps_fill_T_OR, 'clr')
#tranform by species
ps_clr_T_MMEA <- microbiome::transform(ps_T_OP_MMEA, 'clr')
ps_clr_T_DLAB <- microbiome::transform(ps_T_DLAB, 'clr')
ps_clr_T_DSTOK <- microbiome::transform(ps_T_DSTOK, 'clr')
ps_clr_T_SINT <- microbiome::transform(ps_T_SINT, 'clr')
#transform healthy samples
ps_clr_T_AH_DLAB <- microbiome::transform(ps_T_AH_DLAB, 'clr')
ps_clr_T_AH_DSTOK <- microbiome::transform(ps_T_AH_DSTOK, 'clr')
ps_clr_T_AH_SINT <- microbiome::transform(ps_T_AH_SINT, 'clr')
#water samples 
ps_clr_W <- microbiome::transform(ps_W, 'clr')
ps_clr_W_OR =subset_samples(ps_clr_W, Experiment=="original")
ps_clr_W_OP =subset_samples(ps_clr_W, Experiment=="oppurtunistic")
#all samples 
ps_merge_OR_clr <- microbiome::transform(ps_merge_OR, 'clr')
#sediment samples 
ps_clr_S <- microbiome::transform(ps_S, 'clr')

In [ ]:
#ordination 
ps.ord_clrT <- ordinate(ps_clr_T_OR, "RDA", "euclidean")
ps.ord_merge_OR_clr = ordinate(ps_merge_OR_clr, "RDA", "euclidean")
#ordination by species
ps.ord_clrT_OP_MMEA <- ordinate(ps_clr_T_MMEA, "RDA", "euclidean")
ps.ord_clr_T_DLAB <- ordinate(ps_clr_T_DLAB, "RDA", "euclidean")
ps.ord_clr_T_DSTOK <- ordinate(ps_clr_T_DSTOK, "RDA", "euclidean")
ps.ord_clr_T_SINT <- ordinate(ps_clr_T_SINT, "RDA", "euclidean")
#ordination healthy samples
ps.ord_clr_T_AH_DLAB<- ordinate(ps_clr_T_AH_DLAB, "RDA", "euclidean")
ps.ord_clr_T_AH_DSTOK<- ordinate(ps_clr_T_AH_DSTOK, "RDA", "euclidean")
ps.ord_clr_T__AH_SINT<- ordinate(ps_clr_T_AH_SINT, "RDA", "euclidean")
#ordinate water samples
ps.ord_clrW <- ordinate(ps_clr_W, "RDA", "euclidean")
ps.ord_clrW_OR <- ordinate(ps_clr_W_OR, "RDA", "euclidean")
#ordinate sediment samples
ps.ord_clrS <- ordinate(ps_clr_S, "RDA", "euclidean")


# Dispersion

In [ ]:
#sediment samples
dis <- vegdist(otu_table(ps_clr_S), method ="euclidean") 
#homogeneity of group dispersions (variances).
mod <- betadisper(dis, sample_data(ps_clr_S)$Zone)
mod

## Permutation test for F
permutest(mod, pairwise = TRUE)

## Tukey's Honest Significant Differences
TukeyHSD(mod)


In [ ]:
#water samples without LK
dis <- vegdist(otu_table(ps_clr_W_OR), method ="euclidean")
#homogeneity of group dispersions (variances).
mod <- betadisper(dis, sample_data(ps_clr_W_OR)$Zone)
mod
## Permutation test for F
permutest(mod, pairwise = TRUE)

## Tukey's Honest Significant Differences
TukeyHSD(mod)


In [ ]:
#tissue w/out MMEA
dis <- vegdist(otu_table(ps_clr_T_OR), method ="euclidean")
#homogeneity of group dispersions (variances).
mod <- betadisper(dis, sample_data(ps_clr_T_OR)$Conditions)
mod
## Permutation test for F
permutest(mod, pairwise = TRUE)

## Tukey's Honest Significant Differences
TukeyHSD(mod)

In [ ]:
#AH DLAB

dis <- vegdist(otu_table(ps_clr_T_AH_DLAB), method ="euclidean")
#homogeneity of group dispersions (variances).
mod <- betadisper(dis, sample_data(ps_clr_T_AH_DLAB)$Zone)
mod
## Permutation test for F
permutest(mod, pairwise = TRUE)

## Tukey's Honest Significant Differences
TukeyHSD(mod)


In [ ]:
#AH DSTO
dis <- vegdist(otu_table(ps_clr_T_AH_DSTOK), method ="euclidean")
#homogeneity of group dispersions (variances).
mod <- betadisper(dis, sample_data(ps_clr_T_AH_DSTOK)$Zone)
mod
## Permutation test for F
permutest(mod, pairwise = TRUE)

## Tukey's Honest Significant Differences
TukeyHSD(mod)


In [ ]:
#AH SINT
dis <- vegdist(otu_table(ps_clr_T_AH_SINT), method ="euclidean")
#homogeneity of group dispersions (variances).
mod <- betadisper(dis, sample_data(ps_clr_T_AH_SINT)$Zone)
mod
## Permutation test for F
permutest(mod, pairwise = TRUE)

## Tukey's Honest Significant Differences
TukeyHSD(mod)


In [ ]:
#All tissue DLAB
dis <- vegdist(otu_table(ps_clr_T_DLAB), method ="euclidean")
#homogeneity of group dispersions (variances).
mod <- betadisper(dis, sample_data(ps_clr_T_DLAB)$Conditions)
mod
## Permutation test for F
permutest(mod, pairwise = TRUE)

## Tukey's Honest Significant Differences
(mod.HSD <- TukeyHSD(mod))


In [ ]:
#All tissue SINT
dis <- vegdist(otu_table(ps_clr_T_SINT), method ="euclidean")
#homogeneity of group dispersions (variances).
mod <- betadisper(dis, sample_data(ps_clr_T_SINT)$Conditions)
mod
## Permutation test for F
permutest(mod, pairwise = TRUE)

## Tukey's Honest Significant Differences
(mod.HSD <- TukeyHSD(mod))


In [ ]:
#All tissue DSTO
dis <- vegdist(otu_table(ps_clr_T_DSTOK), method ="euclidean")
#homogeneity of group dispersions (variances).
mod <- betadisper(dis, sample_data(ps_clr_T_DSTOK)$Conditions)
mod
## Permutation test for F
permutest(mod, pairwise = TRUE)

## Tukey's Honest Significant Differences
mod.HSD <- TukeyHSD(mod)


In [ ]:
#All tissue MMEA
dis <- vegdist(otu_table(ps_clr_T_MMEA), method ="euclidean")
#homogeneity of group dispersions (variances).
mod <- betadisper(dis, sample_data(ps_clr_T_MMEA)$Condition)
mod
## Permutation test for F
permutest(mod, pairwise = TRUE)

## Tukey's Honest Significant Differences
mod.HSD <- TukeyHSD(mod)


# PERMANOVA 
## Group comparisons

In [ ]:
#convert meta data to dataframe
df_merge_OR_clr_meta= as(sample_data(ps_merge_OR), "data.frame")
#All samples
adonis(otu_table(ps_merge_OR)~ Type, 
       data =df_merge_OR_clr_meta, permutations = 999, 
         method = "euclidean") 

In [ ]:
#convert meta data and counts table to dataframe 
df_ps_clr_W_OR=as.data.frame(otu_table(ps_clr_W_OR))
df_ps_clr_W_OR_meta= as(sample_data(ps_clr_W_OR), "data.frame")

df_ps_clr_S=as.data.frame(otu_table(ps_clr_S))
df_ps_clr_S_meta= as(sample_data(ps_clr_S), "data.frame")

In [ ]:
#water w/out looekey
adonis(formula = otu_table(ps_clr_W_OR) ~ ReefName, data =df_ps_clr_W_OR_meta, permutations = 999, 
         method = "euclidean", strata = df_ps_clr_W_OR_meta$Zone) 
#all sediments
adonis(formula = otu_table(ps_clr_S) ~ ReefName, data =df_ps_clr_S_meta, permutations = 999, 
         method = "euclidean", strata = df_ps_clr_S_meta$Zone) 

In [ ]:
#convert meta data and counts table to dataframe 
df_clr_T_OR_AH_DLAB=as.data.frame(otu_table(ps_clr_T_AH_DLAB))
df_clr_T_OR_meta_AH_DLAB= as(sample_data(ps_clr_T_AH_DLAB), "data.frame")

df_clr_T_OR_AH_DSTOK=as.data.frame(otu_table(ps_clr_T_AH_DSTOK))
df_clr_T_OR_meta_AH_DSTOK= as(sample_data(ps_clr_T_AH_DSTOK), "data.frame")

df_clr_T_OR_AH_SINT=as.data.frame(otu_table(ps_clr_T_AH_SINT))
df_clr_T_OR_meta_AH_SINT= as(sample_data(ps_clr_T_AH_SINT), "data.frame")

In [ ]:
#AH
adonis(otu_table(ps_clr_T_AH_DLAB) ~ Zone, 
       data =df_clr_T_OR_meta_AH_DLAB, permutations = 999, 
         method = "euclidean") 

adonis(otu_table(ps_clr_T_AH_DSTOK) ~ Zone, 
       data = df_clr_T_OR_meta_AH_DSTOK, permutations = 999, 
         method = "euclidean") 


adonis(otu_table(ps_clr_T_AH_SINT) ~ Zone, 
       data =df_clr_T_OR_meta_AH_SINT, permutations = 999, 
         method = "euclidean") 

In [ ]:
#convert meta data and counts table to dataframe 
df_clr_T=as.data.frame(otu_table(ps_clr_T_OR))
df_clr_T_meta= as(sample_data(ps_clr_T_OR), "data.frame")

df_clr_T_OP_MMEA=as.data.frame(otu_table(ps_clr_T_MMEA))
df_clr_T_meta_OP_MMEA= as(sample_data(ps_clr_T_MMEA), "data.frame")


df_clr_T_OR_DLAB=as.data.frame(otu_table(ps_clr_T_DLAB))
df_clr_T_OR_meta_DLAB= as(sample_data(ps_clr_T_DLAB), "data.frame")

df_clr_T_OR_DSTOK=as.data.frame(otu_table(ps_clr_T_DSTOK))
df_clr_T_OR_meta_DSTOK= as(sample_data(ps_clr_T_DSTOK), "data.frame")

df_clr_T_OR_SINT=as.data.frame(otu_table(ps_clr_T_SINT))
df_clr_T_OR_meta_SINT= as(sample_data(ps_clr_T_SINT), "data.frame")

In [ ]:
#all tissue per species - test condition
adonis(df_clr_T ~Conditions,
               data = df_clr_T_meta, permutations=999, method = "euclidian")


adonis(df_clr_T_OR_DLAB ~ Conditions, 
       data =df_clr_T_OR_meta_DLAB, permutations = 999, 
         method = "euclidean") 

adonis(otu_table(ps_clr_T_DSTOK) ~ Conditions, 
       data = df_clr_T_OR_meta_DSTOK, permutations = 999, 
         method = "euclidean") 


adonis(otu_table(ps_clr_T_SINT) ~ Conditions, 
       data =df_clr_T_OR_meta_SINT, permutations = 999, 
         method = "euclidean") 


adonis(otu_table(ps_clr_T_MMEA) ~ Conditions, 
       data =df_clr_T_meta_OP_MMEA, permutations = 999, 
         method = "euclidean") 


In [ ]:
# test 5 tissues and species interactions
df_ps_clr_T= as(sample_data(ps_clr_T_OR), "data.frame")

adonis(otu_table(ps_clr_T_OR)~ Conditions, 
       data =df_ps_clr_T, permutations = 999, 
         method = "euclidean") 


adonis(otu_table(ps_clr_T_OR)~ Conditions*SpeciesCode, 
       data =df_ps_clr_T, permutations = 999, 
         method = "euclidean") 



## Pairwise comparison

In [ ]:
#compare zones in sediment ans water samples
pairwise.adonis(otu_table(ps_clr_S), df_ps_clr_W_OR_meta$Zone, sim.method = "euclidean",
                p.adjust.m = "bonferroni")


pairwise.adonis(otu_table(ps_clr_W_OR), df_ps_clr_W_OR_meta$Zone, sim.method = "euclidean",
                p.adjust.m = "bonferroni")

In [ ]:
#AH compare zones
pairwise.adonis(df_clr_T_OR_AH_DLAB, df_clr_T_OR_meta_AH_DLAB$Zone, sim.method = "euclidean",
                p.adjust.m = "bonferroni",)



pairwise.adonis(df_clr_T_OR_AH_DSTOK, df_clr_T_OR_meta_AH_DSTOK$Zone, sim.method = "euclidean",
                p.adjust.m = "bonferroni",)


pairwise.adonis(df_clr_T_OR_AH_SINT, df_clr_T_OR_meta_AH_SINT$Zone, sim.method = "euclidean",
                p.adjust.m = "bonferroni",)





In [ ]:
#compare VuAH, EnAH, DU, DL
pairwise.adonis(df_clr_T, df_clr_T_meta$Conditions, sim.method = "euclidean",
                p.adjust.m = "bonferroni",)

pairwise.adonis(otu_table(ps_clr_T_OR), df_ps_clr_T$Conditions, 
                sim.method = "euclidean",
                p.adjust.m = "bonferroni")

pairwise.adonis(df_clr_T_OR_DLAB, df_clr_T_OR_meta_DLAB$Conditions, sim.method = "euclidean",
                p.adjust.m = "bonferroni")



pairwise.adonis(df_clr_T_OR_DSTOK, df_clr_T_OR_meta_DSTOK$Conditions, sim.method = "euclidean",
                p.adjust.m = "bonferroni")


pairwise.adonis(df_clr_T_OR_SINT, df_clr_T_OR_meta_SINT$Conditions, sim.method = "euclidean",
                p.adjust.m = "bonferroni")

pairwise.adonis(df_clr_T_OP_MMEA, df_clr_T_meta_OP_MMEA$Conditions, sim.method = "euclidean",
                p.adjust.m = "bonferroni")


In [ ]:
R.version